Kenny Haworth

011694826

**House Generation via Answer Set Programming**

In this assignment, you will be writing a constraint satisfaction program in AnsProlog and solving it with `clingo` -- a piece of Potassco, the Potsdam Answer Set Solving Collection, a bundle of tools for Answer Set Programming developed at the University of Potsdam. 

This will involve writing a program that:

* Infers facts about the state of the world (e.g. is a location safe for placing a wall given the constraints listed below)
* Makes choices about how and where to place a house
* Constrains how the house should be built (e.g. it should not place two doors next to each other)

Answer Set Programming is a declarative programming paradigm, in opposition to the more common imperative programming paradigm.  In an imperative programming paradigm, you tell the computer what to do.  In a declarative programming paradigm, you tell the computer what your desired result is, and then it finds a solution that matches your prescribed solution.  The most common declarative programming language is SQL -- wherein users don't tell the database how to find the results they want, they instead tell the program what kind of results they want and SQL does it.

This notebook exists solely for running `clingo` and printing the results in a pretty manner.  The logic of the program exist in `world_gen.lp` that generates a map based on a set of constraints.  Go over there now and get a feel for the code. 

*
*
*

Now that you have seen the ASP program, here are some helpers defined for you:

`solve` runs a `clingo` sub process, places the results in a file called `dump.lp` (that you can use for debugging), and the processes it into a Python dictionary

`parse_json_result` processes the result as a json file and does the processing into the dictionary file

`pretty_print` takes the resultant dictionary and produces a text grid visualization using emoji

In [1]:
import json
import collections
import subprocess
import random
import sys


def solve(args):
    """Run clingo with the provided argument list and return the parsed JSON result."""

    print_args = ['clingo'] + list(args) + [' | tr [:space:] \\\\n | sort ']
    args = ['clingo', '--outf=2'] + args + ["--sign-def=rnd","--seed="+str(random.randint(0,1<<30))]
    with subprocess.Popen(
        ' '.join(args),
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        shell=True
    ) as clingo:
        outb, err = clingo.communicate()
    if err:
        print(err)
    out = outb.decode("utf-8")
    with open('dump.lp', 'w') as outfile:
        result = json.loads(out)
        witness = result['Call'][0]['Witnesses'][-1]['Value']
        for atom in sorted(witness):
            outfile.write(atom + '\n')
    
    return parse_json_result(out)


def parse_json_result(out):
    """Parse the provided JSON text and extract a dict
    representing the predicates described in the first solver result."""

    result = json.loads(out)

    assert len(result['Call']) > 0
    assert len(result['Call'][0]['Witnesses']) > 0

    witness = result['Call'][0]['Witnesses'][0]['Value']

    class identitydefaultdict(collections.defaultdict):
        def __missing__(self, key):
            return key

    preds = collections.defaultdict(set)
    env = identitydefaultdict()

    for atom in witness:
        if '(' in atom:
            left = atom.index('(')
            functor = atom[:left]
            arg_string = atom[left:]
            try:
                preds[functor].add( eval(arg_string, env) )
            except TypeError:
                pass # at least we tried...

        else:
            preds[atom] = True
    return dict(preds)


def pretty_print(world):
    import functools
    width = functools.reduce(lambda x,y: max(x,y),world['width'])
    height = functools.reduce(lambda x,y: max(x,y),world['height'])
    grid = [[ ' ' for w in range(width)] for h in range(height)]

    legend = {'grass':'🌿',
    'flower':'🌼',
    'bridge':'🌉',
    'water':'🌊',
    'tree':'🌲',
    'door':'🚪',
    'wall':'🗄️',
    'bed':'🛏️',
    'floor':'🔲'}


    for t in world['terrain']:
        xx = int(t[0])
        yy = int(t[1])
        type = t[2]
        grid[yy-1][xx-1] = legend[type]
    for t in world.get('construction',[]):
        xx = int(t[0])
        yy = int(t[1])
        type = t[2]
        grid[yy-1][xx-1] = legend[type]
        
    return ('\n'.join([''.join(r) for r in grid]))

Now we can run it and print the results!

Note: the use of `-c` lets us change a `#const` defined in the ASProlog code, so that we can dynamically resize the resultant map without needing to modify our code.

1. Try decreasing the size to below 10 by 10 -- What happens?
2. Try increasing the size.  At what point does it become too burdensome to wait for you?


In [2]:
# %%time

# width = 12
# height = 12
# world = solve(['world_gen.swi','-c','max_width={}'.format(width),'-c','max_height={}'.format(height),'-t','4'])

# print(pretty_print(world))


Now its your turn!

You will add code to generate a house in your own file `house_gen.swi` .  If you run your code at the same time as `world_gen.swi`, it will generate the terrain at the same time as your house is generated (guaranteeing that the terrain that is generated is capable of supporting your house).

Your code will create a `construction` predicate of the form:

`construction(X_Position, Y_Position, Construction_Type)`

where `Construction_Type` is one of `door, wall, floor, bed`.

This is a free-form assignment, but you have a few constraints that you should satisfy:

* Your house should contain at least 1 door, 1 bed, all non-bed tiles should be floor, and the perimeter of the house should be walls
* It should not place 2 doors side by side
* Doors shouldn't be placed in corners, but should be able to face out to any cardinal direction
* Doors should open to grass or flowers, not water, bridges, or trees
* All pieces of your house should be on top of grass or flowers, not water, bridges, or trees
* The position of your house shouldn't be hardcoded
* A wall piece must touch (in cardinal directions) only 2 doors or walls

* Bonus credit: 10 points -- Your houses can be of variable width and height
* Bonus credit: 10 points -- Your houses can be in shapes that aren't just rectangles

In [4]:
#this code block is the simplest form the constraints

width = 12
height = 12
world = solve(['world_gen.swi','house_gen_simplest_set.swi','-c','max_width={}'.format(width),'-c','max_height={}'.format(height),'-t','4'])

print(pretty_print(world))

🌿🌿🌿🌿🌿🌿🌿🌊🌊🌊🌿🌿
🌿🌿🌿🌿🌿🌿🌊🌊🌊🌊🌊🌿
🌿🌿🌊🌊🌿🌿🌊🌊🌊🌊🌊🌿
🌿🌊🌊🌊🌊🌊🌊🌊🌊🌊🌿🌿
🌿🌊🌊🌊🌊🌿🌉🌉🌉🌿🌿🌿
🌿🌿🌊🌊🌊🌊🌊🌊🌊🌊🌿🌿
🌿🌿🌉🌉🌉🌿🌊🌊🌊🌊🌊🌿
🌿🌿🌊🌊🌼🌿🌿🌊🌊🌊🌊🌿
🌿🌊🌊🌊🌊🌿🌿🌊🌊🌊🌊🌿
🌿🌊🌊🌊🌊🗄️🗄️🗄️🌊🌊🌿🌿
🌲🌿🌊🌊🌿🗄️🛏️🚪🌿🌿🌿🌿
🌿🌿🌿🌿🌿🗄️🗄️🗄️🌿🌿🌿🌿
